In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Caminhos para os arquivos CSV
paths = {
    "Loja 1": "loja_1.csv",
    "Loja 2": "loja_2.csv",
    "Loja 3": "loja_3.csv",
    "Loja 4": "loja_4.csv",
}

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("ç", "c")
        .str.replace("ã", "a")
        .str.replace("á", "a")
        .str.replace("â", "a")
        .str.replace("é", "e")
        .str.replace("ê", "e")
        .str.replace("í", "i")
        .str.replace("ó", "o")
        .str.replace("ô", "o")
        .str.replace("ú", "u")
        .str.replace("ü", "u")
        .str.replace("ï", "i")
    )
    return df

def find_col(df, options):
    for c in options:
        if c in df.columns:
            return c
    return None

lojas = {}
for loja, path in paths.items():
    df = pd.read_csv(path)
    df = normalize_columns(df)
    produto_col = find_col(df, ["produto", "nome_produto", "item"])
    categoria_col = find_col(df, ["categoria", "categoria_produto", "segmento"])
    preco_col = find_col(df, ["preco", "preço", "valor_unitario", "preco_unitario", "price"])
    qtd_col = find_col(df, ["quantidade", "qtd", "qtde", "quantity"])
    frete_col = find_col(df, ["frete", "custo_frete", "valor_frete", "shipping"])
    avaliacao_col = find_col(df, ["avaliacao", "avaliacao_cliente", "nota", "rating"])

    if produto_col:
        df.rename(columns={produto_col: "produto"}, inplace=True)
    if categoria_col:
        df.rename(columns={categoria_col: "categoria"}, inplace=True)
    if preco_col:
        s = df[preco_col].astype(str).str.replace(r"\.", "", regex=True).str.replace(",", ".", regex=False)
        df["preco"] = pd.to_numeric(s, errors="coerce").fillna(0.0)
    else:
        df["preco"] = 0.0
    if qtd_col:
        df["quantidade"] = pd.to_numeric(df[qtd_col], errors="coerce").fillna(0).astype(float)
    else:
        df["quantidade"] = 1.0
    if frete_col:
        s = df[frete_col].astype(str).str.replace(r"\.", "", regex=True).str.replace(",", ".", regex=False)
        df["frete"] = pd.to_numeric(s, errors="coerce").fillna(0.0)
    else:
        df["frete"] = np.nan
    if avaliacao_col:
        df["avaliacao"] = pd.to_numeric(df[avaliacao_col], errors="coerce")
    else:
        df["avaliacao"] = np.nan

    df["receita"] = df["preco"] * df["quantidade"]
    lojas[loja] = df


In [ ]:
resumo = []
for loja, df in lojas.items():
    faturamento = df["receita"].sum()
    cat_pop = df.groupby("categoria")["quantidade"].sum().sort_values(ascending=False).index[0] if "categoria" in df.columns else None
    media_av = df["avaliacao"].mean(skipna=True)
    frete_medio = df["frete"].mean(skipna=True)
    vendas_prod = df.groupby("produto")["quantidade"].sum().sort_values(ascending=False)
    produto_mais = vendas_prod.index[0]
    produto_menos = vendas_prod.index[-1]
    resumo.append({
        "Loja": loja,
        "Faturamento": faturamento,
        "Categoria_mais_popular": cat_pop,
        "Media_avaliacao": media_av,
        "Frete_medio": frete_medio,
        "Produto_mais_vendido": produto_mais,
        "Produto_menos_vendido": produto_menos
    })

resumo_df = pd.DataFrame(resumo)
resumo_df


In [ ]:
# 1. Faturamento por Loja
plt.bar(resumo_df["Loja"], resumo_df["Faturamento"])
plt.title("Faturamento por Loja")
plt.xlabel("Loja")
plt.ylabel("Faturamento")
plt.show()

# 2. Participação no Faturamento Total
plt.pie(resumo_df["Faturamento"], labels=resumo_df["Loja"], autopct="%1.1f%%", startangle=90)
plt.title("Participação no Faturamento Total")
plt.show()

# 3. Média de Avaliação por Loja
plt.bar(resumo_df["Loja"], resumo_df["Media_avaliacao"])
plt.title("Média de Avaliação por Loja")
plt.xlabel("Loja")
plt.ylabel("Média de Avaliação")
plt.show()


# Relatório – Análise de Desempenho das Lojas
## Objetivo
O Sr. João, proprietário de quatro lojas da rede Alura Store, deseja vender uma delas para investir em um novo negócio.  
O objetivo desta análise é identificar qual loja apresenta o pior desempenho, utilizando métricas de vendas e operação.

## Métricas analisadas
- **Faturamento total** por loja
- **Categorias mais populares** (por quantidade vendida)
- **Produtos mais e menos vendidos**
- **Custo médio do frete**
- *(Média de avaliação: sem dados disponíveis na base)*

## Resultados principais

| Loja   | Faturamento (R$) | Categoria mais popular | Produto mais vendido |
|--------|------------------|------------------------|----------------------|
| Loja 1 | 141.592.863,00   | [ver análise detalhada] | [ver análise detalhada] |
| Loja 2 | 136.677.888,00   | [ver análise detalhada] | [ver análise detalhada] |
| Loja 3 | 133.131.850,00   | [ver análise detalhada] | [ver análise detalhada] |
| **Loja 4** | **125.654.773,00**   | [ver análise detalhada] | [ver análise detalhada] |

📊 **Resumo:**
- A **Loja 4** apresenta **o menor faturamento** e a menor participação no faturamento total da rede.
- O custo médio de frete da Loja 4 é o mais baixo, mas este fator não compensa seu baixo desempenho em vendas.

## Conclusão
Com base nas métricas, é recomendado vender a Loja 4, pois:
1. Apresenta o menor faturamento.
2. Tem a menor participação no faturamento total.
3. Apesar do frete médio mais baixo, não compensa a perda de receita.

**Recomendação final:** vender a Loja 4.
